In [1]:
import PIL
import numpy as np
import torch
import torchvision
from torchvision.transforms.v2 import Compose, GaussianBlur, RandomEqualize, RandomSolarize, RandomApply
import wandb
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from Dataset.AerialDataset import AerialDataset
from tasks.SRDiffTrainer import SRDiffTrainer
from models.SRDIFFBuilder import SRDiffBuilder
from utils.model_utils import load_model

#Data
lr_size = 64
hr_size = 256
batch_size = 20
dataset_dir = 'C:\\Users\\adrianperera\\Desktop\\dataset_tfg'

transforms = Compose(
    [RandomApply(transforms= [GaussianBlur(7)], p = 0.5),
    RandomEqualize()]
)

dataset = AerialDataset(dataset_dir, lr_size, hr_size, data_augmentation = None, aux_sat_prob= 0.4, sat_dataset_path= "C:\\Users\\adrianperera\\Desktop\\dataset_tfg\\satelite_dataset\\64_256")
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6, 0.2, 0.2], generator=torch.Generator().manual_seed(420))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
device = torch.device(0)

In [3]:
world_size = torch.cuda.device_count()
hyperparams = {
    "lr": 0.00002,
    "model_name": f"SRDiff Percp loss",
    "epochs": 200,
    "eta_min": 1e-7,
    "num_workers":torch.cuda.device_count(),
    "mode": "min",
    "patience": 5,
    "factor": 0.1,
    "model": "SRDiff",
    "batch_size": 20,
    "ddp": True,
    "grad_acum": 1,
    "use_rrdb": True,
    "fix_rrdb": False,
    "aux_l1_loss": False,
    "aux_perceptual_loss": True,
    "aux_ssim_loss": False,
    "losstype": "l1"
}

model_builder = SRDiffBuilder()
model_builder = model_builder.set_standart()
model_builder = model_builder.set_losstype(hyperparams["losstype"], hyperparams["aux_l1_loss"], hyperparams["aux_perceptual_loss"])
model = model_builder.build()
model_data = model_builder.get_hyperparameters()
model.to(device)
trainer = SRDiffTrainer(metrics_used=("ssim", "psnr"), model_name=hyperparams["model_name"],
                            device=device, use_rrdb=hyperparams["use_rrdb"],
                            fix_rrdb=hyperparams["fix_rrdb"], aux_ssim_loss=hyperparams["aux_ssim_loss"],
                            aux_l1_loss=hyperparams["aux_l1_loss"],
                            aux_perceptual_loss=hyperparams["aux_perceptual_loss"])

model = load_model(model, "SRDiff Percp loss199.pt", "saved models\\SRDiff\\Distributed\\Version 2")

trainer.set_model(model)

wandb.login()
wandb.init(project="SRDiff experiments", config=hyperparams.update(model_data), name=f"{hyperparams["model_name"]} version 2 testing",
               group=f"{hyperparams["model_name"]} version 2 ddp group")

test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)

C:\Users\adrianperera\miniconda3\envs\cuda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\adrianperera\miniconda3\envs\cuda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


The model weights have been loaded from 'saved models\SRDiff\Distributed\Version 2\SRDiff Percp loss199.pt'


psnr,▁
ssim,▁
psnr,17.86586
ssim,0.55544


100%|██████████| 40/40 [08:18<00:00, 12.45s/batch, psnr=6.39, ssim=-0.307]


In [3]:
model_name = "SRDIFF standart version 1.5"
hyperparams = {
    "lr":0.00002,
    "epochs":100,
    "eta_min":1e-7,
    "decay_steps": 1000000,
    "mode": "min",
    "factor":0.5,
    "model" : "SRDiff",
    "batch_size" : batch_size,
    "ddp": False,
    "grad_acum": 1,
    "use_rrdb":True,
    "fix_rrdb":True,
    "aux_l1_loss":False,
    "aux_perceptual_loss":False,
    "aux_ssim_loss":False
}
hyperparams.update(model_builder.get_hyperparameters())
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams["lr"])
params = list(model.named_parameters())
if not hyperparams['fix_rrdb']:
    params = [p for p in params if 'rrdb' not in p[0]]
params = [p[1] for p in params]
optimizer =  torch.optim.Adam(params, lr=hyperparams['lr'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hyperparams["decay_steps"], gamma=hyperparams["factor"])
torch.backends.cudnn.benchmark = True
project_name = "SRDiff experiments"
run_name = model_name
wandb.login()
wandb.init(project=project_name, config=hyperparams, name=run_name)

trainer = SRDiffTrainer(metrics_used=("ssim", "psnr"), model_name=model_name, device=device, 
                        use_rrdb=hyperparams["use_rrdb"], fix_rrdb=hyperparams["fix_rrdb"], aux_ssim_loss=hyperparams["aux_ssim_loss"],
                        aux_l1_loss=hyperparams["aux_l1_loss"], aux_perceptual_loss=hyperparams["aux_perceptual_loss"],
                        grad_acum=hyperparams["grad_acum"])

trainer.set_model(model)
trainer.set_optimizer(optimizer)
trainer.set_scheduler(scheduler)
for step in range(hyperparams["epochs"]):
    with torch.no_grad():
        val_loss = trainer.validate(val_dataloader)
    torch.cuda.empty_cache()
    train_loss = trainer.train(train_dataloader, step)
    torch.cuda.empty_cache()
    if step % 10 == 0:
        trainer.save_model("saved models\\SRDiff\\depured tests\\")
    torch.cuda.empty_cache()
    wandb.log({"train_loss": train_loss, "validation_loss": val_loss})
    
test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)
wandb.finish()

In [4]:
model_builder = SRDiffBuilder()
model = model_builder.set_standart().build()
model.to(device)

model_name = "SRDIFF standart version 2.5"
hyperparams = {
    "lr":0.00002,
    "epochs":100,
    "eta_min":1e-7,
    "decay_steps": 1000000,
    "mode": "min",
    "factor":0.5,
    "model" : "SRDiff",
    "batch_size" : batch_size,
    "ddp": False,
    "grad_acum": 1,
    "use_rrdb":True,
    "fix_rrdb":True,
    "aux_l1_loss":True,
    "aux_perceptual_loss":False,
    "aux_ssim_loss":False
}
hyperparams.update(model_builder.get_hyperparameters())
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams["lr"])
params = list(model.named_parameters())
if not hyperparams['fix_rrdb']:
    params = [p for p in params if 'rrdb' not in p[0]]
params = [p[1] for p in params]
optimizer =  torch.optim.Adam(params, lr=hyperparams['lr'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hyperparams["decay_steps"], gamma=hyperparams["factor"])
torch.backends.cudnn.benchmark = True
project_name = "SRDiff experiments"
run_name = model_name
wandb.login()
wandb.init(project=project_name, config=hyperparams, name=run_name)

trainer = SRDiffTrainer(metrics_used=("ssim", "psnr"), model_name=model_name, device=device, 
                        use_rrdb=hyperparams["use_rrdb"], fix_rrdb=hyperparams["fix_rrdb"], aux_ssim_loss=hyperparams["aux_ssim_loss"],
                        aux_l1_loss=hyperparams["aux_l1_loss"], aux_perceptual_loss=hyperparams["aux_perceptual_loss"],
                        grad_acum=hyperparams["grad_acum"])

trainer.set_model(model)
trainer.set_optimizer(optimizer)
trainer.set_scheduler(scheduler)
for step in range(hyperparams["epochs"]):
    with torch.no_grad():
        val_loss = trainer.validate(val_dataloader)
    torch.cuda.empty_cache()
    train_loss = trainer.train(train_dataloader, step)
    torch.cuda.empty_cache()
    if step % 10 == 0:
        trainer.save_model("saved models\\SRDiff\\depured tests\\")
    torch.cuda.empty_cache()
    wandb.log({"train_loss": train_loss, "validation_loss": val_loss})
    
test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)
wandb.finish()

In [5]:
model_builder = SRDiffBuilder()
model = model_builder.set_standart().build()
model.to(device)

model_name = "SRDIFF standart version 3.5"
hyperparams = {
    "lr":0.00002,
    "epochs":100,
    "eta_min":1e-7,
    "decay_steps": 1000000,
    "mode": "min",
    "factor":0.5,
    "model" : "SRDiff",
    "batch_size" : batch_size,
    "ddp": False,
    "grad_acum": 1,
    "use_rrdb":True,
    "fix_rrdb":True,
    "aux_l1_loss":True,
    "aux_perceptual_loss":False,
    "aux_ssim_loss":True
}
hyperparams.update(model_builder.get_hyperparameters())
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams["lr"])
params = list(model.named_parameters())
if not hyperparams['fix_rrdb']:
    params = [p for p in params if 'rrdb' not in p[0]]
params = [p[1] for p in params]
optimizer =  torch.optim.Adam(params, lr=hyperparams['lr'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hyperparams["decay_steps"], gamma=hyperparams["factor"])
torch.backends.cudnn.benchmark = True
project_name = "SRDiff experiments"
run_name = model_name
wandb.login()
wandb.init(project=project_name, config=hyperparams, name=run_name)

trainer = SRDiffTrainer(metrics_used=("ssim", "psnr"), model_name=model_name, device=device, 
                        use_rrdb=hyperparams["use_rrdb"], fix_rrdb=hyperparams["fix_rrdb"], aux_ssim_loss=hyperparams["aux_ssim_loss"],
                        aux_l1_loss=hyperparams["aux_l1_loss"], aux_perceptual_loss=hyperparams["aux_perceptual_loss"],
                        grad_acum=hyperparams["grad_acum"])

trainer.set_model(model)
trainer.set_optimizer(optimizer)
trainer.set_scheduler(scheduler)
for step in range(hyperparams["epochs"]):
    with torch.no_grad():
        val_loss = trainer.validate(val_dataloader)
    torch.cuda.empty_cache()
    train_loss = trainer.train(train_dataloader, step)
    torch.cuda.empty_cache()
    if step % 10 == 0:
        trainer.save_model("saved models\\SRDiff\\depured tests\\")
    torch.cuda.empty_cache()
    wandb.log({"train_loss": train_loss, "validation_loss": val_loss})
    
test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)
wandb.finish()

In [6]:
from models.SR3Builder import SR3Builder
from tasks.SR3Trainer import SR3Trainer


lr_size = 64
hr_size = 256
batch_size = 128
dataset_dir = 'C:\\Users\\adrianperera\\Desktop\\dataset_tfg'

dataset = AerialDataset(dataset_dir, lr_size, hr_size, data_augmentation = None, aux_sat_prob= 0.5, sat_dataset_path= "C:\\Users\\adrianperera\\Desktop\\dataset_tfg\\satelite_dataset\\64_256")
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6, 0.2, 0.2], generator=torch.Generator().manual_seed(420))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
device = torch.device(0)

model_builder = SR3Builder()
model_builder = model_builder.set_standart()
model_builder = model_builder.set_sample_steps(1000)
model_builder = model_builder.set_losstype("l1")
model = model_builder.build()
model.to(device)

hyperparams = {
    "lr":0.0002,
    "epochs":100,
    "eta_min":1e-7,
    "decay_steps": 100000,
    "gamma" : 0.5,  
    "model" : "SR3",
    "grad_acum": 0,
    "ddp": False,
    "batch_size":batch_size
}
hyperparams.update(model_builder.get_hyperparameters())
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams["lr"])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hyperparams["decay_steps"], gamma=hyperparams["gamma"])

project_name = "SR model benchmarking"
run_name = "SR3 standart l1, 128Batch, 100k sampling"
wandb.login()
wandb.init(project=project_name, config=hyperparams, name=run_name)

trainer = SR3Trainer(metrics_used=("ssim", "psnr"), model_name="SR3 Standart l1 DA 128b 100k sampling", grad_acum=hyperparams["grad_acum"])
trainer.set_model(model)
trainer.set_optimizer(optimizer)
trainer.set_scheduler(scheduler)
for epoch in range(hyperparams["epochs"]):  
    with torch.no_grad():
        val_loss = trainer.validate(val_dataloader)
    
    train_loss = trainer.train(train_dataloader, epoch)
    torch.cuda.empty_cache()
    if epoch % 10 == 0:
        trainer.save_model("saved models\\SR3")

    torch.cuda.empty_cache()
    wandb.log({"train_loss": train_loss, "validation_loss": val_loss})
    
test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)
wandb.finish()   